In [ ]:
# price_prediction.ipynb

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import requests

# Fetch historical price data for Solana
price_url = "https://api.coingecko.com/api/v3/coins/solana/market_chart?vs_currency=usd&days=30"
response = requests.get(price_url)
data = response.json()

# Prepare the data
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Feature engineering (using time-based features for simplicity)
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['price_diff'] = df['price'].diff()

# Drop any NaN values
df = df.dropna()

# Train a regression model to predict future prices
X = df[['hour', 'dayofweek', 'price_diff']]  # Features
y = df['price']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model: Linear Regression (or use a more advanced model as needed)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Plot predictions
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test.values, label="True Prices")
plt.plot(y_test.index, y_pred, label="Predicted Prices")
plt.title("Solana Price Prediction")
plt.xlabel("Time")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

# Save the model and predictions
import joblib
joblib.dump(model, 'price_prediction_model.pkl')
df.to_csv('price_prediction_results.csv', index=False)
